In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression

import plotly.graph_objects as go

import numpy as np


In [2]:
def to_int(value):
    try:
        n = value.strip()
        n = n.replace(",",'')
        return int(n)
    except:
        return value
    
def to_pct(value):
    try:
        n = value.replace("%",'')
        return float(n)
    except:
        return value
    
    
def to_date(value):
    return pd.to_datetime(value)

def predictions(column):
    column = column.dropna()
    data = column.values
    model = ARIMA(data, order=(1, 1, 1))
    model_fit = model.fit(disp=False,transparams=False)
    yhat = model_fit.predict(len(data),len(data)+3, typ='levels')
    return yhat

In [24]:
mhi = pd.read_csv('data/Median_Household_Income.csv')
mhv = pd.read_csv('data/Median_Home_Values.csv')
eighty = pd.read_csv('data/80_Pop.csv')

In [25]:
cities = eighty.CBSA.to_list()

In [26]:
drop = ['CBSA Code','CBSA']

In [27]:
eighty = eighty.drop(drop,axis=1)
eighty = eighty.T
eighty.columns = cities

In [40]:
mhv = mhv.drop(drop,axis=1)
mhv = mhv.T
mhv.columns = cities

mhi = mhi.drop(drop,axis=1)
mhi = mhi.T
mhi.columns = cities



In [47]:
files = [mhi, mhv, eighty]
names = ['Median_Household_Income','Median_Home_Values','80_Pop']

In [59]:
delta = {}
for city in cities:
    data = []
    for each in files:
        single = each[city].apply(lambda x: to_int(x))[-8:]
        X = np.array(single.keys()).reshape(-1, 1)
        y = single.values
        reg = LinearRegression().fit(X, y)
        data.append(float(reg.coef_))
    delta[city] = data

In [60]:
summary = pd.DataFrame.from_dict(delta, orient='index')
summary.columns = names

In [75]:
summary.sort_values('Median_Household_Income', ascending=False).head(10)

,Median_Household_Income,Median_Home_Values,80_Pop
"San Jose, CA",2753.321429,64023.428571,1947.952381
"San Francisco, CA",2363.273810,14901.190476,4685.226190
"Washington, DC",1940.416667,4838.559524,7185.619048
"Los Angeles, CA",1748.142857,12918.714286,14100.333333
"San Diego, CA",1642.452381,10229.369048,2584.928571
"Seattle, WA",1633.369048,3794.654762,3616.226190
"Boston, MA",1574.047619,3452.809524,3908.119048
"Houston, TX",1562.857143,1078.892857,7417.678571
"Baltimore, MD",1555.916667,2141.285714,2067.595238
"New York, NY",1474.250000,4355.976190,17843.392857


In [73]:
summary.sort_values('Median_Home_Values', ascending=False).head(10)

,Median_Household_Income,Median_Home_Values,80_Pop
"San Jose, CA",2753.321429,64023.428571,1947.952381
"San Francisco, CA",2363.273810,14901.190476,4685.226190
"Los Angeles, CA",1748.142857,12918.714286,14100.333333
"San Diego, CA",1642.452381,10229.369048,2584.928571
"Sacramento, CA",1184.321429,5049.511905,2465.309524
"Washington, DC",1940.416667,4838.559524,7185.619048
"New York, NY",1474.250000,4355.976190,17843.392857
"Portland, OR",1084.440476,4105.988095,2551.654762
"Seattle, WA",1633.369048,3794.654762,3616.226190
"Riverside, CA",858.226190,3682.547619,5080.619048


In [72]:
summary.sort_values('80_Pop', ascending=False).head(10)

,Median_Household_Income,Median_Home_Values,80_Pop
"New York, NY",1474.250000,4355.976190,17843.392857
"Los Angeles, CA",1748.142857,12918.714286,14100.333333
"Chicago, IL",975.130952,830.726190,8747.892857
"Dallas, TX",1271.714286,1599.607143,8254.404762
"Phoenix, AZ",786.797619,2526.476190,8037.321429
"Miami, FL",923.595238,2094.833333,7861.428571
"Houston, TX",1562.857143,1078.892857,7417.678571
"Washington, DC",1940.416667,4838.559524,7185.619048
"Atlanta, GA",823.392857,667.559524,7017.142857
"Riverside, CA",858.226190,3682.547619,5080.619048
